## Load Data for Retail Churn Demo ##

### Note - Be sure to populate bucket name and any required keys in vars.json ###
1. **demo_ofs.retail_compnew**. User Event History - OFS
2. **demo_ofs.web_comment**.  Customer Comment History - OFS
3. **demo.retail_customer**. Customer Dimension information - BFS

In [1]:
import teradatasql, json

# load vars json
with open('../../vars.json', 'r') as f:
    session_vars = json.load(f)

name = session_vars['hierarchy']['SYSDBA']['username']
pwd = session_vars['hierarchy']['SYSDBA']['password']

bucket = session_vars['environment']['bucket']

with teradatasql.connect(host = session_vars['environment']['host'], 
                     user = name, 
                     password = pwd) as con:
    cur = con.cursor()
    try:
        cur.execute('DROP TABLE demo_ofs.retail_compnew;')
    except Exception as e:
        # Table already exists
        if str(e.args).find("3807") >= 1:
            pass
        else:
            raise
            
    qry = '''
    CREATE MULTISET TABLE demo_ofs.retail_compnew
   
    (CUSTOMER_ID DECIMAL(18,0),
    DATESTAMP TIMESTAMP(6),
    EVENT VARCHAR(50) CHARACTER SET LATIN CASESPECIFIC);
    '''
    cur.execute(qry)
    
    qry = f'''
    Insert into demo_ofs.retail_compnew 
    SELECT CUSTOMER_ID, DATESTAMP, EVENT
    FROM
            (
                LOCATION = '/s3/{bucket}.s3.amazonaws.com/DEMO_OFS/RETAIL_COMPNEW/manifest.json'
                MANIFEST = 'TRUE'
                AUTHORIZATION = Repositories.PubAuth
            ) D;'''
    cur.execute(qry)

In [2]:
import teradatasql, json

# load vars json
with open('../../vars.json', 'r') as f:
    session_vars = json.load(f)

name = session_vars['hierarchy']['SYSDBA']['username']
pwd = session_vars['hierarchy']['SYSDBA']['password']

bucket = session_vars['environment']['bucket']

with teradatasql.connect(host = session_vars['environment']['host'], 
                     user = name, 
                     password = pwd) as con:
    cur = con.cursor()
    try:
        cur.execute('DROP TABLE demo_ofs.web_comment;')
    except Exception as e:
        # Table already exists
        if str(e.args).find("3807") >= 1:
            pass
        else:
            raise
            
    qry = '''
    CREATE MULTISET TABLE demo_ofs.web_comment
    
    (comment_id INTEGER,
    customer_id INTEGER,
    comment_text VARCHAR(50000) CHARACTER SET LATIN CASESPECIFIC,
    comment_summary VARCHAR(50000) CHARACTER SET LATIN CASESPECIFIC);
    '''
    cur.execute(qry)
    
    qry = f'''
    Insert into demo_ofs.web_comment
    SELECT comment_id, customer_id, comment_text, comment_summary
    FROM
            (
                LOCATION = '/s3/{bucket}.s3.amazonaws.com/DEMO_OFS/WEB_COMMENT/manifest.json'
                MANIFEST = 'TRUE'
                AUTHORIZATION = Repositories.PubAuth
            ) D;'''
    cur.execute(qry)

In [3]:
import teradatasql, json

# load vars json
with open('../../vars.json', 'r') as f:
    session_vars = json.load(f)

name = session_vars['hierarchy']['SYSDBA']['username']
pwd = session_vars['hierarchy']['SYSDBA']['password']

with teradatasql.connect(host = session_vars['environment']['host'], 
                     user = name, 
                     password = pwd) as con:
    cur = con.cursor()
    try:
        cur.execute('DROP TABLE demo.retail_customer;')
    except Exception as e:
        # Table already exists
        if str(e.args).find("3807") >= 1:
            pass
        else:
            raise
            
    qry = '''
    CREATE MULTISET TABLE demo.retail_customer
    
    (CUSTOMER_ID DECIMAL(18,0) NOT NULL,
    F_NAME VARCHAR(30) CHARACTER SET LATIN CASESPECIFIC,
    L_NAME VARCHAR(30) CHARACTER SET LATIN CASESPECIFIC,
    VALIDITY PERIOD(DATE) FORMAT 'YYYY-MM-DD' AS VALIDTIME,
    CUST_ZIP VARCHAR(5) CHARACTER SET LATIN CASESPECIFIC,
    CUST_LOCATION SYSUDTLIB.ST_GEOMETRY(16776192) INLINE LENGTH 9920,
    ETHNICITY VARCHAR(20) CHARACTER SET LATIN CASESPECIFIC,
    GENDER CHAR(1) CHARACTER SET LATIN CASESPECIFIC,
    INCOME INTEGER,
    AGE INTEGER,
    CHURN_FLAG VARCHAR(1) CHARACTER SET LATIN CASESPECIFIC)
    
    PRIMARY INDEX ( CUSTOMER_ID );
    '''
    cur.execute(qry)
    
    qry = f'''
    Insert into demo.retail_customer
    SELECT CUSTOMER_ID,
    F_NAME,
    L_NAME,
    VALIDITY,
    CUST_ZIP,
    NEW ST_GEOMETRY("CUST_LOCATION") AS CUST_LOCATION,
    ETHNICITY,
    GENDER,
    INCOME,
    AGE,
    CHURN_FLAG
    FROM
            (
                LOCATION = '/s3/{bucket}.s3.amazonaws.com/DEMO/RETAIL_CUSTOMER/manifest.json'
                MANIFEST = 'TRUE'
                AUTHORIZATION = Repositories.PubAuth
            ) D;'''
    cur.execute(qry)
